# Обучение определения фейковых фактов о COVID и вакцинации

In [1]:
import math

import torch
import pandas as pd
import numpy as np

In [2]:
DEVICE = 'cpu'
if torch.backends.mps.is_available():
    DEVICE = 'mps'
if torch.cuda.is_available():
    DEVICE = 'cuda'

In [3]:
from pathlib import Path

DATA_PATH = Path('data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = Path('data/cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = Path('data/models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

pd.set_option('display.max_colwidth', 500) 

In [4]:
MODEL_NAME = "covid_vaccine_fake_model"
TEST_DF_NAME = "facebook_data_to_model.xlsx"

MAX_LENGTH = 128
BATCH_SIZE = 64

# Датасет

In [ ]:
data_df = pd.read_excel(DATA_PATH / TEST_DF_NAME)
data_df.head(1)

In [7]:
idx2label = {
    0: "Real",
    1: "Fake"
}

In [ ]:
from typing import Dict, Tuple

import torch
from torch.utils.data import Dataset
from transformers import PreTrainedTokenizer
import pandas as pd


class TokenizedDataset(Dataset):
    def __init__(
        self,
        dataframe: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        max_length: int,
        text_column: str = "text",
        tensor_dtype: Tuple[torch.dtype, torch.dtype] = (torch.long, torch.long),
    ):
        """
        Инициализация датасета с токенизацией.

        Args:
            dataframe (pd.DataFrame): DataFrame с колонкой, содержащей текст для токенизации.
            tokenizer (PreTrainedTokenizer): Токенайзер для преобразования текста.
            max_length (int): Максимальная длина токенов.
            text_column (str): Название колонки с текстом.
            tensor_dtype (tuple): Типы данных для токенов (input_ids, attention_mask).
        """
        self.tensor_dtype = tensor_dtype

        if text_column not in dataframe.columns:
            raise ValueError(f"Колонка '{text_column}' отсутствует в DataFrame")

        tokenized_data = tokenizer(
            dataframe[text_column].tolist(),
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        self.input_ids = tokenized_data["input_ids"].to(dtype=self.tensor_dtype[0])
        self.attention_mask = tokenized_data["attention_mask"].to(dtype=self.tensor_dtype[1])

    def __len__(self) -> int:
        """
        Возвращает количество примеров в датасете.
        """
        return len(self.input_ids)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Возвращает токенизированные данные.

        Args:
            idx (int): Индекс примера.

        Returns:
            Dict[str, torch.Tensor]: Словарь с токенами.
        """
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }


In [ ]:
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizer
import pandas as pd


def create_dataloader(
    dataframe: pd.DataFrame,
    tokenizer: PreTrainedTokenizer,
    text_column: str = "text",
    max_length: int = 64,
    batch_size: int = 16,
    shuffle: bool = False,
    tensor_dtype=(torch.long, torch.long),
) -> DataLoader:
    """
    Создание DataLoader из DataFrame без меток (для предсказания).

    Args:
        dataframe (pd.DataFrame): DataFrame с колонкой текста.
        tokenizer (PreTrainedTokenizer): Токенайзер для преобразования текста.
        text_column (str): Название колонки с текстом.
        max_length (int): Максимальная длина токенов.
        batch_size (int): Размер батча.
        shuffle (bool): Перемешивать ли данные (по умолчанию False).
        tensor_dtype (tuple): Типы данных для токенов.

    Returns:
        DataLoader: DataLoader для предсказания.
    """
    dataset = TokenizedDataset(dataframe, tokenizer, max_length, text_column=text_column, tensor_dtype=tensor_dtype)
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
    )
    return dataloader


In [ ]:
from transformers import RobertaTokenizer

tokenizer: RobertaTokenizer = RobertaTokenizer.from_pretrained(DATA_PATH_SAVE_MODELS / MODEL_NAME)

test_loader_1 = create_dataloader(
    dataframe=data_df,
    tokenizer=tokenizer,
    text_column="text",
    max_length=MAX_LENGTH,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_loader_2 = create_dataloader(
    dataframe=data_df,
    tokenizer=tokenizer,
    text_column="text",
    max_length=MAX_LENGTH,
    batch_size=BATCH_SIZE,
    shuffle=False
)


# Модель

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model = RobertaForSequenceClassification.from_pretrained(
    DATA_PATH_SAVE_MODELS / MODEL_NAME)

model.to(DEVICE)

# Тестирование

In [19]:
import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import pandas as pd


def run_model_on_dataloader(
    model: torch.nn.Module,
    dataloader: DataLoader,
    device: torch.device
) -> tuple:
    """
    Прогоняет модель по даталоадеру и возвращает предсказания и вероятности.

    Args:
        model (torch.nn.Module): Обученная модель.
        dataloader (DataLoader): DataLoader для обработки.
        device (torch.device): Устройство для вычислений (CPU/GPU).

    Returns:
        Tuple: (список предсказанных меток, список вероятностей)
    """
    model.eval()
    model.to(device)

    predictions = []
    probabilities = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Processing"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Получаем логиты
            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

            # Добавляем в списки
            predictions.extend(preds)
            probabilities.extend(probs.tolist())

    return predictions, probabilities


def test_model_on_dataset(
    model: torch.nn.Module,
    dataloader_1: DataLoader,
    dataloader_2: DataLoader,
    test_df: pd.DataFrame,
    idx2label: dict,
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
) -> pd.DataFrame:
    """
    Прогоняет датасет через модель для двух даталоадеров и добавляет результаты в DataFrame.

    Args:
        model (torch.nn.Module): Обученная модель.
        dataloader_1 (DataLoader): Первый DataLoader.
        dataloader_2 (DataLoader): Второй DataLoader.
        test_df (pd.DataFrame): Исходный DataFrame (без меток).
        idx2label (dict): Словарь, отображающий индексы категорий в названия.
        device (torch.device): Устройство для вычислений (CPU/GPU).

    Returns:
        pd.DataFrame: DataFrame с добавленными предсказаниями:
            - 'predict_1': предсказанная метка для первого даталоадера,
            - 'probability_1': вероятность предсказания для первого даталоадера,
            - 'predict_2': предсказанная метка для второго даталоадера,
            - 'probability_2': вероятность предсказания для второго даталоадера.
    """
    print("Processing DataLoader 1...")
    predictions_1, probabilities_1 = run_model_on_dataloader(model, dataloader_1, device)

    print("Processing DataLoader 2...")
    predictions_2, probabilities_2 = run_model_on_dataloader(model, dataloader_2, device)

    # Преобразуем предсказанные индексы в метки
    test_df = test_df.reset_index(drop=True)
    test_df["predict_1"] = [idx2label[p] for p in predictions_1]
    test_df["probability_1"] = probabilities_1
    test_df["predict_2"] = [idx2label[p] for p in predictions_2]
    test_df["probability_2"] = probabilities_2

    return test_df


In [ ]:

test_results_df = test_model_on_dataset(
    model=model,
    dataloader_1=test_loader_1,
    dataloader_2=test_loader_2,
    test_df=data_df,
    idx2label=idx2label,
    device=DEVICE,
)

In [ ]:
test_results_df.sample(10)